In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from filterpy.kalman import ExtedKalmanFilter
from filterpy.common import Q_discrete_white_noise
import random

In [ ]:
def ybusppg(num):
## Creation of y bus
    fb = linedata7(:,1)     # From bus number...
    tb = linedata7(:,2)     # To bus number...
    r = linedata7(:,3)      # Resistance, R...
    x = linedata7(:,4)      # Reactance, X...
    b = linedata7(:,5)      # Ground Admittance, B/2...
    a = linedata7(:,6)      # Tap setting value..
    z = r + x*1j            # Z matrix...
    y = 1/z               # To get inverse of each element...
    b = b*1j                # Make B imaginary...

    nbus = max(max(fb),max(tb))    # no. of buses...
    nbranch = length(fb)           # no. of branches...
    ybus = zeros(nbus,nbus)        # Initialise YBus...
    
    # Formation of the Off Diagonal Elements...
    for k=1:nbranch
        ybus(fb(k),tb(k)) = ybus(fb(k),tb(k))-y(k)/a(k)
        ybus(tb(k),fb(k)) = ybus(fb(k),tb(k))
    
    
    # Formation of Diagonal Elements....
    for m =1:nbus
        for n =1:nbranch
            if fb(n) == m
                ybus(m,m) = ybus(m,m) + y(n)/(a(n)^2) + b(n)
            elseif tb(n) == m
                ybus(m,m) = ybus(m,m) + y(n) + b(n)
            
        
    

In [3]:

#           |Msnt |Type | Value | From |   To |   Rii | 
zdata7= [ #---- Voltage Magnitude ------------#
            [1,    1,    0.9927,   1,       0,   9e-4],
            # #-----------------------------------#
            # #---- Real Power Injection ---------#
            [2,    2,    0.8109,   3,       0,   1e-4],
            [3,    2,    0.9695,   5,       0,   1e-4], 
            [4,    2,    2.0565,   7,       0,   1e-4],
           #------------------------------------#
           #---- Reative Power Injection -------#
            [5,     3,    0.4214,   3,       0,   1e-4],
            [6,     3,   -0.8890,   5,       0,   1e-4], 
            [7,     3,    0.3889,   7,       0,   1e-4],
           #------------------------------------#
           #------ Real Power Flow ------------- #
            [8,     4,   -1.6884,   1,       2,   64e-6],
            [9,     4,   -1.5799,   2,       3,   64e-6],
           [10,     4,    0.9292,   3,       4,   64e-6],
           [11,     4,   -1.7369,   3,       7,   64e-6],
           [12,     4,   -1.1166,   2,       6,   64e-6],
           [13,     4,   -1.3063,   6,       5,   64e-6],
           [14,     4,    0.4093,   7,       6,   64e-6],
           #------------------------------------#
           #------ Reactive Power Flow ---------#   
           [15,     5,    1.8083,   1,       2,   64e-6],
           [16,     5,    0.5710,   2,       3,   64e-6],
           [17,     5,    0.4367,   3,       4,   64e-6],
           [18,     5,   -0.3230,   3,       7,   64e-6],
           [19,     5,    0.8677,   2,       6,   64e-6],
           [20,     5,    1.4774,   6,       5,   64e-6],
           [21,     5,    0.8325,   7,       6,   64e-6]]

In [6]:

def  linedatas(num):
    ## 
    #         |  From |  To   |   R     |   X     |     B/2  |  X'mer  |
    #         |  Bus  | Bus   |  pu     |  pu     |     pu   | TAP (a) |
    linedata7=[  [1,      2,       0.07149,   0.17840,       0.0,       1],
                [2,      3,       0.00596,   0.01487,       0.0,       1],
                [2,      6,       0.01291,   0.03221,       0.0,       1],
                [3,      4,       0.02284,   0.05699,       0.0,       1],
                [3,      7,       0.00496,   0.01239,       0.0,       1],
                [6,      5,       0.01688,   0.04209,       0.0,       1],
                [6,      7,       0.01390,   0.03469,       0.0,       1]]
    
    return linedata7

In [ ]:
def HJacobian_at1(x):



    num = nbus 
    #ybus = ybusppg(num) # Get YBus..nope we already have it
    zdata = zdata7 # Get Measurement data..
        ##
    fb = linedata7(:,1)
    tb = linedata7(:,2)
    b = linedata7(:,5)
    nbus = max(max(fb),max(tb))    # no. of buses...
    nbranch = length(fb)           # no. of branches...
    bbus = zeros(nbus,nbus)

    for k=1:nbranch
        bbus(fb(k),tb(k)) = b(k)
        bbus(tb(k),fb(k)) = bbus(fb(k),tb(k))
    
    ##
    bpq = bbus # Get B data..
    #nbus = max(max(zdata(:,4)),max(zdata(:,5))) # Get number of buses..
    type = zdata7(:,2) # Type of measurement, Vi - 1, Pi - 2, Qi - 3, Pij - 4, Qij - 5, Iij - 6..
    z = zdata7(:,3) # Measuement values..
    fbus = zdata7(:,4) # From bus..
    tbus = zdata7(:,5) # To bus..
    Ri = diag(zdata7(:,6)) # Measurement Error..
    V = ones(nbus,1) # Initialize the bus voltages..
    del = zeros(nbus,1) # Initialize the bus angles..
    E = [del(2:) V]   # State Vector..
    G = real(ybus)
    B = imag(ybus)

    vi = find(type == 1) # Index of voltage magnitude measurements..
    ppi = find(type == 2) # Index of real power injection measurements..
    qi = find(type == 3) # Index of reactive power injection measurements..
    pf = find(type == 4) # Index of real powerflow measurements..
    qf = find(type == 5) # Index of reactive powerflow measurements..

    nvi = length(vi) # Number of Voltage measurements..
    npi = length(ppi) # Number of Real Power Injection measurements..
    nqi = length(qi) # Number of Reactive Power Injection measurements..
    npf = length(pf) # Number of Real Power Flow measurements..
    nqf = length(qf) # Number of Reactive Power Flow measurements..
    # # H11 - Derivative of V with respect to angles.. All Zeros
    H11 = zeros(nvi,nbus-1)

    # H12 - Derivative of V with respect to V.. 
    H12 = zeros(nvi,nbus)
    for k = 1:nvi
        for n = 1:nbus
            if n == k
                H12(k,n) = 1
            
        
    

    # H21 - Derivative of Real Power Injections with Angles..
    H21 = zeros(npi,nbus-1)
    for i = 1:npi
        m = fbus(ppi(i))
        for k = 1:(nbus-1)
            if k+1 == m
                for n = 1:nbus
                    H21(i,k) = H21(i,k) + V(m)* V(n)*(-G(m,n)*sin(del(m)-del(n)) + B(m,n)*cos(del(m)-del(n)))
                
                H21(i,k) = H21(i,k) - V(m)^2*B(m,m)
            else
                H21(i,k) = V(m)* V(k+1)*(G(m,k+1)*sin(del(m)-del(k+1)) - B(m,k+1)*cos(del(m)-del(k+1)))
            
        
    
    
    # H22 - Derivative of Real Power Injections with V..
    H22 = zeros(npi,nbus)
    for i = 1:npi
        m = fbus(ppi(i))
        for k = 1:(nbus)
            if k == m
                for n = 1:nbus
                    H22(i,k) = H22(i,k) + V(n)*(G(m,n)*cos(del(m)-del(n)) + B(m,n)*sin(del(m)-del(n)))
                
                H22(i,k) = H22(i,k) + V(m)*G(m,m)
            else
                H22(i,k) = V(m)*(G(m,k)*cos(del(m)-del(k)) + B(m,k)*sin(del(m)-del(k)))
            
        
    
    
    # H31 - Derivative of Reactive Power Injections with Angles..
    H31 = zeros(nqi,nbus-1)
    for i = 1:nqi
        m = fbus(qi(i))
        for k = 1:(nbus-1)
            if k+1 == m
                for n = 1:nbus
                    H31(i,k) = H31(i,k) + V(m)* V(n)*(G(m,n)*cos(del(m)-del(n)) + B(m,n)*sin(del(m)-del(n)))
                
                H31(i,k) = H31(i,k) - V(m)^2*G(m,m)
            else
                H31(i,k) = V(m)* V(k+1)*(-G(m,k+1)*cos(del(m)-del(k+1)) - B(m,k+1)*sin(del(m)-del(k+1)))
            
        
    
    
    # H32 - Derivative of Reactive Power Injections with V..
    H32 = zeros(nqi,nbus)
    for i = 1:nqi
        m = fbus(qi(i))
        for k = 1:(nbus)
            if k == m
                for n = 1:nbus
                    H32(i,k) = H32(i,k) + V(n)*(G(m,n)*sin(del(m)-del(n)) - B(m,n)*cos(del(m)-del(n)))
                
                H32(i,k) = H32(i,k) - V(m)*B(m,m)
            else
                H32(i,k) = V(m)*(G(m,k)*sin(del(m)-del(k)) - B(m,k)*cos(del(m)-del(k)))
            
        
    
    
    # H41 - Derivative of Real Power Flows with Angles..
    H41 = zeros(npf,nbus-1)
    for i = 1:npf
        m = fbus(pf(i))
        n = tbus(pf(i))
        for k = 1:(nbus-1)
            if k+1 == m
                H41(i,k) = V(m)* V(n)*(-G(m,n)*sin(del(m)-del(n)) + B(m,n)*cos(del(m)-del(n)))
            else 
                if k+1 == n
                H41(i,k) = -V(m)* V(n)*(-G(m,n)*sin(del(m)-del(n)) + B(m,n)*cos(del(m)-del(n)))
                else
                    H41(i,k) = 0
                
            
        
    
    
    # H42 - Derivative of Real Power Flows with V..
    H42 = zeros(npf,nbus)
    for i = 1:npf
        m = fbus(pf(i))
        n = tbus(pf(i))
        for k = 1:nbus
            if k == m
                H42(i,k) = -V(n)*(-G(m,n)*cos(del(m)-del(n)) - B(m,n)*sin(del(m)-del(n))) - 2*G(m,n)*V(m)
            else 
                if k == n
                H42(i,k) = -V(m)*(-G(m,n)*cos(del(m)-del(n)) - B(m,n)*sin(del(m)-del(n)))
                else
                    H42(i,k) = 0
                
            
        
    
    
    # H51 - Derivative of Reactive Power Flows with Angles..
    H51 = zeros(nqf,nbus-1)
    for i = 1:nqf
        m = fbus(qf(i))
        n = tbus(qf(i))
        for k = 1:(nbus-1)
            if k+1 == m
                H51(i,k) = -V(m)* V(n)*(-G(m,n)*cos(del(m)-del(n)) - B(m,n)*sin(del(m)-del(n)))
            else 
                if k+1 == n
                H51(i,k) = V(m)* V(n)*(-G(m,n)*cos(del(m)-del(n)) - B(m,n)*sin(del(m)-del(n)))
                else
                    H51(i,k) = 0
                
            
        
    
    
    # H52 - Derivative of Reactive Power Flows with V..
    H52 = zeros(nqf,nbus)
    for i = 1:nqf
        m = fbus(qf(i))
        n = tbus(qf(i))
        for k = 1:nbus
            if k == m
                H52(i,k) = -V(n)*(-G(m,n)*sin(del(m)-del(n)) + B(m,n)*cos(del(m)-del(n))) - 2*V(m)*(-B(m,n)+ bpq(m,n))
            else 
                if k == n
                H52(i,k) = -V(m)*(-G(m,n)*sin(del(m)-del(n)) + B(m,n)*cos(del(m)-del(n)))
                else
                    H52(i,k) = 0
                
            
        
    
    
    # Measurement Jacobian, H..
    H = [H11 H12 H21 H22 H31 H32 H41 H42 H51 H52]